# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('C:/Users/212313601/Documents/Coursera/Machine Learning Specialization - Regression/kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to andylofgreen@gmail.com and will expire on July 15, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\212313601\AppData\Local\Temp\graphlab_server_1499740984.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
1e10

10000000000.0

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.367137     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.403140     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [7]:
model_all.coefficients[model_all.coefficients['value']>0]

name,index,value,stderr
(intercept),None,274873.05595,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
grade,None,842.068034898,None
sqft_above,None,20.0247224171,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

# Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
import numpy as np

In [10]:
l1_list = np.logspace(1,7,num=13)

In [11]:
rssList = []
for l1 in l1_list:
    model_train = graphlab.linear_regression.create(training, 
                                                  target='price', 
                                                  features=all_features,
                                                  validation_set=None, 
                                                  l2_penalty=0., 
                                                  l1_penalty=l1,
                                                  verbose=False)
    predicted = model_train.predict(validation)
    residuals = predicted - validation['price']
    rss = (residuals**2).sum()
    rssList.append(rss)
    

In [12]:
rssList

[625766285142459.9,
 625766285362394.1,
 625766286057885.0,
 625766288257224.6,
 625766295212186.8,
 625766317206080.5,
 625766386760658.1,
 625766606749278.5,
 625767302791634.1,
 625769507643886.2,
 625776517727024.0,
 625799062845467.0,
 625883719085425.2]

In [13]:
import pandas as pd
df1=pd.DataFrame(rssList,columns=['rss'])
df2=pd.DataFrame(l1_list,columns=['l1_penalty'])

In [14]:
df=df1.join(df2)
df.sort(columns=['rss'],ascending=True)

C:\Users\212313601\AppData\Local\Continuum\Anaconda3\envs\gl-env\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,rss,l1_penalty
0,6.257663e+14,1.000000e+01
1,6.257663e+14,3.162278e+01
2,6.257663e+14,1.000000e+02
3,6.257663e+14,3.162278e+02
4,6.257663e+14,1.000000e+03
5,6.257663e+14,3.162278e+03
6,6.257664e+14,1.000000e+04
7,6.257666e+14,3.162278e+04
8,6.257673e+14,1.000000e+05
9,6.257695e+14,3.162278e+05


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [15]:
l1_list[0]

10.0

In [16]:
1.0e7

10000000.0

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [17]:
model_train = graphlab.linear_regression.create(training, 
                                                target='price', 
                                                features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., 
                                                l1_penalty=l1_list[0],
                                                verbose=False)
model_train.coefficients.print_rows(100)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [18]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [19]:
l1_penalty_values1 = np.logspace(8, 10, num=20)

In [20]:
l1_penalty_values1

array([  1.00000000e+08,   1.27427499e+08,   1.62377674e+08,
         2.06913808e+08,   2.63665090e+08,   3.35981829e+08,
         4.28133240e+08,   5.45559478e+08,   6.95192796e+08,
         8.85866790e+08,   1.12883789e+09,   1.43844989e+09,
         1.83298071e+09,   2.33572147e+09,   2.97635144e+09,
         3.79269019e+09,   4.83293024e+09,   6.15848211e+09,
         7.84759970e+09,   1.00000000e+10])

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [21]:
nonzero_list = []
for l1 in l1_penalty_values1:
    model_train = graphlab.linear_regression.create(training, 
                                                    target='price', 
                                                    features=all_features,
                                                    validation_set=None, 
                                                    l2_penalty=0., 
                                                    l1_penalty=l1,
                                                    verbose=False)
    nonzeros = model_train['coefficients']['value'].nnz()
    nonzero_tuple = (nonzeros,l1)
    nonzero_list.append(nonzero_tuple)

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [22]:
nonzero_list

[(18, 100000000.0),
 (18, 127427498.57031322),
 (18, 162377673.91887242),
 (18, 206913808.11147901),
 (17, 263665089.87303555),
 (17, 335981828.62837881),
 (17, 428133239.8719396),
 (17, 545559478.11685145),
 (17, 695192796.17755914),
 (16, 885866790.41008317),
 (15, 1128837891.6846883),
 (15, 1438449888.2876658),
 (13, 1832980710.8324375),
 (12, 2335721469.0901213),
 (10, 2976351441.6313128),
 (6, 3792690190.7322536),
 (5, 4832930238.5717525),
 (3, 6158482110.6602545),
 (1, 7847599703.5146227),
 (1, 10000000000.0)]

In [33]:
import math
l1_penalty_min = nonzero_list[14][1]
l1_penalty_max = nonzero_list[15][1]
print(l1_penalty_min)
print(math.ceil(l1_penalty_min/1000000)*1000000)
print(l1_penalty_max)
print(math.ceil(l1_penalty_max/1000000)*1000000)

2976351441.63
2977000000.0
3792690190.73
3793000000.0


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [24]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [25]:
rssList = []
for l1 in l1_penalty_values:
    model_train = graphlab.linear_regression.create(training, 
                                                    target='price', 
                                                    features=all_features,
                                                    validation_set=None, 
                                                    l2_penalty=0., 
                                                    l1_penalty=l1,
                                                    verbose=False)
    if model_train['coefficients']['value'].nnz() ==7:
        predicted = model_train.predict(validation)
        residuals = predicted - validation['price']
        rss = (residuals**2).sum()
        rss_l1_tuple = (rss,l1)
        rssList.append(rss_l1_tuple)
    else:
        rss  = 0
        rss_l1_tuple = (rss,l1)
        rssList.append(rss_l1_tuple)

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [26]:
rssList

[(0, 2976351441.6313128),
 (0, 3019316638.952415),
 (0, 3062281836.2735171),
 (0, 3105247033.5946193),
 (0, 3148212230.9157214),
 (0, 3191177428.2368236),
 (0, 3234142625.5579257),
 (0, 3277107822.8790278),
 (0, 3320073020.20013),
 (0, 3363038217.5212321),
 (0, 3406003414.8423343),
 (1046937488751711.1, 3448968612.1634364),
 (1051147625612860.9, 3491933809.4845386),
 (1055992735342999.1, 3534899006.8056407),
 (1060799531763287.8, 3577864204.1267428),
 (0, 3620829401.447845),
 (0, 3663794598.7689471),
 (0, 3706759796.0900493),
 (0, 3749724993.4111514),
 (0, 3792690190.7322536)]

In [27]:
model_train = graphlab.linear_regression.create(training, 
                                                    target='price', 
                                                    features=all_features,
                                                    validation_set=None, 
                                                    l2_penalty=0., 
                                                    l1_penalty=3448968612.1634364,
                                                    verbose=False)
model_train['coefficients'].print_rows(100)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |